![](https://www.gminsights.com/assets/img/all-terrain-vehicle-atv-market-pressrelease.png)gminsights.com

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import random
import os
import cv2
import sys
from pylab import rcParams
from PIL import Image
from tqdm import tqdm
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#Codes by Junyi Ng https://www.kaggle.com/junyingsg/step-by-step-guide-to-denoising-your-labels

In [ ]:
#Seed everything for reproducible results
import random
from numpy.random import seed
from tensorflow.random import set_seed

seed_value = 42
random.seed(seed_value)
seed(seed_value)
set_seed(seed_value)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, Flatten, Dropout, Activation, Input, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.mixed_precision import experimental as mixed_precision
from sklearn.model_selection import StratifiedKFold

In [ ]:
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy) #shortens training time by 2x

In [ ]:
gps = pd.read_csv("../input/offroad-terrain-dataset-for-autonomous-vehicles/SensorData/2020-10-02-10-17-05/gps.csv")
gps.head()

In [ ]:
df_train = pd.read_csv("../input/offroad-terrain-dataset-for-autonomous-vehicles/ImageLabels/tsm_1_labels.csv")
df_train.head()

In [ ]:
df2_train = pd.read_csv("../input/offroad-terrain-dataset-for-autonomous-vehicles/ImageLabels/tsm_2_labels.csv")
df2_train.head()

In [ ]:
#df_train["tsm1_original"] = df_train["tsm1_original"].astype(str) #convert to str as we want to use cross entropy loss later
df_train.info()

#Image Augmentation

In [ ]:
batch_size=32
image_size=300

input_shape = (image_size, image_size, 3)
target_size = (image_size, image_size)
img_augmentation = tf.keras.Sequential(
    [
        tf.keras.layers.experimental.preprocessing.RandomCrop(image_size, image_size),
        tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
        tf.keras.layers.experimental.preprocessing.RandomRotation(0.25),
        tf.keras.layers.experimental.preprocessing.RandomContrast(0.2)
    ])

In [ ]:
path = "../input/offroad-terrain-dataset-for-autonomous-vehicles/Images/Images/2020-09-24/"
files = df_train["image"].tolist()
file = random.choice(files)
image = Image.open("../input/offroad-terrain-dataset-for-autonomous-vehicles/Images/Images/2020-09-24/969904752s168ms.jpg")
plt.imshow(image)
plt.axis("off")
plt.show()

#Augmentation helps the model generalize better. What images can look like after augmentation.

In [ ]:
image = tf.expand_dims(np.array(image), 0)

plt.figure(figsize=(14, 14))
for i in range(9):
    augmented_image = img_augmentation(image)
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(augmented_image[0])
    plt.axis("off")

In [ ]:
#df_train["tsm1_original"] = df_train["tsm1_original"].astype(str) #convert to str as we want to use cross entropy loss later

In [ ]:
def DataGenerator(train_set, val_set):
    
    train_datagen = ImageDataGenerator().flow_from_dataframe(
                  dataframe = train_set,
                  directory='../input/offroad-terrain-dataset-for-autonomous-vehicles/Images/Images/',
                  x_col='image',
                  y_col='tsm1_original',
                  target_size=target_size,
                  batch_size=batch_size,
                  shuffle=True,
                  class_mode='sparse',
                  seed=seed_value)

    val_datagen = ImageDataGenerator().flow_from_dataframe(
                dataframe = val_set,
                directory='../input/offroad-terrain-dataset-for-autonomous-vehicles/Images/Images/',
                x_col='image',
                y_col='tsm1_original',
                target_size=target_size,
                batch_size=batch_size,
                shuffle=False,
                class_mode='sparse',
                seed=seed_value)
    
    return train_datagen, val_datagen

In [ ]:
epochs = 3
total_steps = (int(len(df_train)*0.8/batch_size)+1)*epochs

lr = tf.keras.experimental.CosineDecay(initial_learning_rate=1e-3, decay_steps=total_steps)

In [ ]:
def build_model():
    base_model = EfficientNetB0(include_top=False, weights="imagenet", input_shape=input_shape)

    # Rebuild top
    inputs = Input(shape=input_shape)
    base = base_model(inputs)
    pooling = GlobalAveragePooling2D()(base)
    outputs = Dense(5, activation="softmax", dtype='float32')(pooling) #necessary for mixed-precision training to work properly

    # Compile
    model = Model(inputs=inputs, outputs=outputs)
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=['accuracy'])
    return model

In [ ]:
fold_number = 1
n_splits = 5
oof_accuracy = []

tf.keras.backend.clear_session()
skf = StratifiedKFold(n_splits=n_splits, random_state=None)
for train_index, val_index in skf.split(df_train["image"], df_train["tsm1_original"]):
    train_set = df_train.loc[train_index]
    val_set = df_train.loc[val_index]
    train_datagen, val_datagen = DataGenerator(train_set, val_set)
    model = build_model()
    print("Training fold no.: " + str(fold_number+1))

    model_name = "effnetb0 "
    fold_name = "fold.h5"
    filepath = model_name + str(fold_number+1) + fold_name
    callbacks = [ModelCheckpoint(filepath=filepath, monitor='val_accuracy', save_best_only=True)]

    history = model.fit(train_datagen, epochs=epochs, validation_data=val_datagen, callbacks=callbacks)
    oof_accuracy.append(max(history.history["val_accuracy"]))
    fold_number += 1
    if fold_number == n_splits:
        print("Training finished!")

#ValueError: Asked to retrieve element 0, but the Sequence has length 0

In [ ]:
#Code by Olga Belitskaya https://www.kaggle.com/olgabelitskaya/sequential-data/comments
from IPython.display import display,HTML
c1,c2,f1,f2,fs1,fs2=\
'#eb3434','#eb3446','Akronim','Smokum',30,15
def dhtml(string,fontcolor=c1,font=f1,fontsize=fs1):
    display(HTML("""<style>
    @import 'https://fonts.googleapis.com/css?family="""\
    +font+"""&effect=3d-float';</style>
    <h1 class='font-effect-3d-float' style='font-family:"""+\
    font+"""; color:"""+fontcolor+"""; font-size:"""+\
    str(fontsize)+"""px;'>%s</h1>"""%string))
    
    
dhtml('Be patient. Marília Prata, @mpwolke Was here' )